# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115998e+02     1.626585e+00
 * time: 0.7473390102386475
     1     1.092973e+01     8.317787e-01
 * time: 2.3838138580322266
     2    -1.188764e+01     9.747492e-01
 * time: 2.9743828773498535
     3    -3.401248e+01     7.295665e-01
 * time: 3.8490920066833496
     4    -4.752021e+01     5.890651e-01
 * time: 4.746306896209717
     5    -5.685637e+01     2.052221e-01
 * time: 5.574563026428223
     6    -5.970965e+01     1.611998e-01
 * time: 6.1446428298950195
     7    -6.087515e+01     6.190715e-02
 * time: 6.726143836975098
     8    -6.135692e+01     4.452040e-02
 * time: 7.298379898071289
     9    -6.160814e+01     5.025029e-02
 * time: 7.878394842147827
    10    -6.180142e+01     2.896235e-02
 * time: 8.478790998458862
    11    -6.195166e+01     2.283339e-02
 * time: 9.057917833328247
    12    -6.204726e+01     1.576440e-02
 * time: 9.635327816009521
    13    -6.209865e+01     1.237451e-02
 * time: 10.210279941558838
 

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671079
    AtomicLocal         -18.8557707
    AtomicNonlocal      14.8522656
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485386 
    Xc                  -19.3336826

    total               -62.261666458462